In [245]:
%reload_ext autoreload
%autoreload 2

In [256]:
from pathlib import Path
import sys
from IPython.display import display, HTML
import bpy
import typst
import tempfile

project_root = Path.home() / "projects/blender_typst_importer/"
sys.path.append(str(project_root))


def display_svg(svg_content, width='100px'):
    html_content = (
        "<style>"
        ".svg-container svg { max-width: 100%; height: auto; display: block; }"
        "</style>"
    )
    html_content += f"<div class='svg-container' style='width:{width};'>{svg_content}</div>"
    display(HTML(html_content))

#uv pip install lxml
#uv pip install numpy==1.26.4


temp_dir = Path(tempfile.gettempdir())
#temp_dir = Path.cwd()
typst_file = temp_dir / "step1.typ"
svg_file = temp_dir / "step1.svg"

In [257]:
file_content = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)

$ a= b/c $  
"""
typst_file.write_text(file_content)
typst.compile(typst_file, format="svg", output=str(svg_file))
#print(svg_file.read_text())

In [258]:
file_content = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)

#let korange() = text(fill: orange)[$k$]
#let nblue() = text(fill: blue)[$n$]
$ sum_(#korange() = 1)^#nblue() #korange() = (nblue()(nblue()+1)) / 2 $  
"""
typst_file.write_text(file_content)
typst.compile(typst_file, format="svg", output=str(svg_file))

In [259]:
file_content = """
#set page(width: 900pt, height: auto, margin: 0cm, fill: none)

#import "@preview/codelst:2.0.1": sourcecode
#import "@preview/tablex:0.0.8": tablex
#let sourcecode = sourcecode.with(frame: (code) => block(
  radius: 5pt,
  stroke: luma(30),
  inset: 30pt,
  text(size: 30pt, code)
))

#sourcecode[```python
for i in range(0,10):
  print(i)
```]
"""
typst_file.write_text(file_content)
typst.compile(typst_file, format="svg", output=str(svg_file))


In [261]:
from typst_importer.typst_to_svg import simplify_svg, replace_stroke_with_path

step1_content = svg_file.read_text()
step2_content = simplify_svg(step1_content)
step3_content = replace_stroke_with_path(step2_content)

display_svg(step1_content , width='500px')
display_svg(step3_content , width='500px')

In [262]:
output_file = temp_dir / "step3.svg"
output_file.write_text(step3_content)

bpy.ops.import_curve.svg(filepath=str(output_file))
col = bpy.context.scene.collection.children['step3.svg']
col.name = "Formula"

for obj in col.objects:
    obj.scale = (100, 100, 100)

In [263]:
# Loop through each object in the collection and set its origin
bpy.ops.object.select_all(action='DESELECT')
if col.objects:
    # Set the first object as active
    bpy.context.view_layer.objects.active = col.objects[0]
    # Now we can safely set the mode to OBJECT
    bpy.ops.object.mode_set(mode='OBJECT') 
    for obj in col.objects:
        bpy.context.view_layer.objects.active = obj
        obj.select_set(True)
        bpy.ops.object.origin_set(type='ORIGIN_GEOMETRY', center='MEDIAN')
        obj.select_set(False)

# Experiment with `grease_pencil_import_svg`

In [264]:
from pathlib import Path
import typst
import tempfile

temp_dir = Path(tempfile.gettempdir())
typst_file = temp_dir / "step1.typ"
svg_file = temp_dir / "step1.svg"

file_content = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)

#let korange() = text(fill: orange)[$k$]
#let nblue() = text(fill: blue)[$n$]

$ sum_(#korange() = 1)^#nblue() #korange() = (nblue()(nblue()+1)) / 2 + B + F + O $  
"""
typst_file.write_text(file_content)
typst.compile(typst_file, format="svg", output=str(svg_file))

from typst_importer.typst_to_svg import simplify_svg, replace_stroke_with_path

step1_content = svg_file.read_text()
step2_content = simplify_svg(step1_content)
step3_content = replace_stroke_with_path(step2_content)

output_file = temp_dir / "output.svg"
output_file.write_text(step3_content)

bpy.ops.wm.grease_pencil_import_svg(filepath=str(output_file))

{'FINISHED'}